In [ ]:
! pip install langchain langchain-google-genai

# Self Consistancy and Multiple Paths of Reasoning
Sometimes LLM produce inconsistant and unrealiable output.By `leveraging multiple reasoning paths and aggregating results`, we can enhance the robustness and accuracy of AI-generated response

In [4]:
import os
import random
from collections import Counter
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ['GOOGLE_API_KEY']='GOOGLE_API_KEY'

llm=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

# Generating Multiple Reasoning Paths

In [5]:
def generate_multiple_paths(problem,num_paths=3):
  """
  Generate a multipe reasoning path for a given problem:

  Args:
    problem(str): The problem statement
    num_paths(int): Number of reasoning paths to generate

  Returns:
    list: A list of generated reasoning paths.
  """
  prompt_template=PromptTemplate(
      input_variables=['problem','path_number'],
      template="""
            Solve the following problem by unique approach. This is reasoning path {path_number}.
            Problem: {problem}
            Reasoning path : {path_number}
      """
  )

  paths=[]
  for i in range(num_paths):
    chain = prompt_template | llm
    response= chain.invoke({"problem":problem,"path_number":i+1}).content
    paths.append(response)

  return paths

In [6]:
# Let's test out function
problem="A ball thrown upwards at initial velocity 20 m/s. How high it will go ?"

paths=generate_multiple_paths(problem)

for i,path in enumerate(paths,1):
  print(f"Path {i}:\n{path}\n")

Path 1:
Reasoning Path 1:  Using Energy Conservation

This approach avoids directly using kinematic equations. Instead, we leverage the principle of conservation of mechanical energy.

1. **Initial Energy:**  At the moment the ball is thrown, its energy is purely kinetic.  The kinetic energy (KE) is given by:

   KE = (1/2) * m * v²

   where:
     * m = mass of the ball (this will cancel out)
     * v = initial velocity = 20 m/s

2. **Maximum Height Energy:** At the ball's highest point, its velocity is momentarily zero. Therefore, its kinetic energy is zero.  All its initial kinetic energy has been converted into potential energy (PE). The potential energy is given by:

   PE = m * g * h

   where:
     * g = acceleration due to gravity (approximately 9.8 m/s²)
     * h = maximum height

3. **Energy Conservation:** Since energy is conserved (ignoring air resistance), the initial kinetic energy equals the potential energy at the maximum height:

   (1/2) * m * v² = m * g * h

4. **Sol

# Aggregating Results

In [7]:
def aggregate_results(paths):
  """
  Aggregate results from multiple reasoning paths.

  Args:
    paths: List of reasoning paths

  Returns:
    List: The most consistant Answer
  """
  prompt_template=PromptTemplate(
      input_variables=['paths'],
      template="""
        Analyze the follwing reasoning paths and determine the most consistant answer. If there are discrepancies,
        Explain why and provide the mostly likely correct answer.
        Reasoning Paths: {paths}
        Most Consistant Answer:
      """
  )
  chain= prompt_template | llm
  response=chain.invoke({'paths':"\n".join(paths)}).content
  return response

In [8]:
aggregated_result=aggregate_results(paths)
print('Aggregated Result:\n',aggregated_result)

Aggregated Result:
 All three reasoning paths are consistent and arrive at the same answer: the ball will reach a maximum height of approximately 20.4 meters.  They all correctly apply the principle of conservation of mechanical energy, where the initial kinetic energy is converted entirely into potential energy at the maximum height.  The slight variations in how the equations are presented are purely stylistic and do not affect the final result.  The mass of the ball cancels out in each calculation, simplifying the solution.  Therefore, the most consistent answer is **20.4 meters**.


# Self Consistency Check


In [11]:
def self_consistency_check(problem,aggregated_result):
  """
  Peform a self-consistency check on aggregated result.

  Args:
    problem(str): Original problem statement
    aggregated_result(str): Aggregated result to check

  Return:
    List: A evalution of result's consistency and reliability.
  """
  prompt_template=PromptTemplate(
      input_variables=['problem','result'],
      template="""
        Evaluate the consistency and realibility of the following result for given problem.
        Problem:{problem}
        Result:{result}

        Evaluation(Consider factors like logical consistency, adherence to known facts, and potential biases):
      """
  )

  chain=prompt_template | llm
  response=chain.invoke({'problem':problem,'result':aggregate_results}).content
  return response

In [12]:
consistency_evaluation = self_consistency_check(problem, aggregated_result)
print("Self-Consistency Evaluation:\n", consistency_evaluation)

Self-Consistency Evaluation:
 The provided result "<function aggregate_results at 0x7da6780f4fe0>" is not a numerical result; it's a Python representation of a function's memory address.  This means no actual calculation was performed, and therefore, we cannot evaluate the consistency and reliability of the *result* itself.  We can only evaluate the *method* used to solve the problem (which is implied to be within the `aggregate_results` function, but we don't know its contents).

To evaluate the *potential* consistency and reliability of a *correct* solution to the problem, we need to consider:

**Logical Consistency and Adherence to Known Facts:**

The problem involves simple projectile motion under the influence of gravity.  A consistent and reliable solution would adhere to these facts:

* **Constant acceleration due to gravity:**  We assume a constant gravitational acceleration (approximately 9.81 m/s² downwards).  This is a reasonable assumption for relatively short distances.
* 

# Applying to Different Problem Type

In [13]:
def solve_problem(problem):
    """
    Solve a problem using multiple reasoning paths, aggregation, and self-consistency check.

    Args:
    problem (str): The problem statement.

    Returns:
    tuple: (aggregated_result, consistency_evaluation)
    """
    paths = generate_multiple_paths(problem)
    aggregated_result = aggregate_results(paths)
    consistency_evaluation = self_consistency_check(problem, aggregated_result)
    return aggregated_result, consistency_evaluation

# Example problems
problems = [
    "What is the capital of France?",
    "Explain the concept of supply and demand in economics.",
    "If a train travels at 60 km/h, how long will it take to cover 180 km?"
]

for problem in problems:
    print(f"Problem: {problem}")
    result, evaluation = solve_problem(problem)
    print("Aggregated Result:\n", result)
    print("\nConsistency Evaluation:\n", evaluation)
    print("\n" + "-"*50 + "\n")

Problem: What is the capital of France?
Aggregated Result:
 The most consistent answer is **Paris**. All three reasoning paths arrive at the same conclusion, although they use slightly different approaches.

There are no significant discrepancies between the reasoning paths.  They all rely on a process of elimination, leveraging different types of prior knowledge:

* **Reasoning Path 1** uses the most common misconception – confusing Paris with other major European capitals.  It's a strong method because it directly addresses the source of potential error.

* **Reasoning Path 2** cleverly uses the *reverse* misconception – the mistaken belief that Paris is the capital of *another* country. While less direct, it still effectively eliminates alternatives.

* **Reasoning Path 3** employs geographical proximity as an additional filter for elimination. This adds another layer of plausibility, making the conclusion even more robust.

While Reasoning Path 2 is slightly less intuitive than Pat